In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import date
from functools import partial
import os

# 저장용 리스트
todo_items = []

# 공통 레이아웃 설정
full_width = widgets.Layout(width='100%')
half_width = widgets.Layout(width='48%')
third_width = widgets.Layout(width='32%')
label_layout = widgets.Layout(width='80px')
input_layout = widgets.Layout(width='100%')
button_layout = widgets.Layout(width='100%', height='36px')

# 전체 진행률
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=widgets.Layout(width='100%', height='20px'))
overall_progress_label = widgets.Label(value="(0%)")

# 리스트 박스
todo_list_box = widgets.VBox(layout=widgets.Layout(border='none', padding='0px'))

# 전체 진행률 업데이트 함수
def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"
    bar_color = f"rgb({255-int(avg*2.55)}, {255-int(avg*2.55)}, {255-int(avg*2.55)})"
    overall_progress_bar.style = {'bar_color': bar_color}

# 리스트 새로 고침
def refresh_list():
    todo_list_box.children = []
    for idx, item in enumerate(todo_items, 1):
        title = widgets.Label(value=f"# {idx}. {item['title']}", layout=full_width)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=full_width)
        bar_color = f"rgb({255-int(item['progress']*2.55)}, {255-int(item['progress']*2.55)}, {255-int(item['progress']*2.55)})"
        progress_bar.style = {'bar_color': bar_color}
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=full_width)
        progress_label = widgets.Label(value=f"{item['progress']}%", layout=widgets.Layout(width='100%', margin='0 0 8px 0'))

        due_label = widgets.Label(value=f"{item['due']}", layout=full_width)
        priority_label = widgets.Label(value=item['priority'], layout=full_width)

        complete_btn = widgets.Button(description='완료✅', layout=half_width)
        delete_btn = widgets.Button(description='삭제❌', layout=half_width)
        button_row = widgets.HBox([complete_btn, delete_btn], layout=full_width)

        def on_progress_change(change, i=item, bar=progress_bar, pl=progress_label):
            i['progress'] = change['new']
            bar.value = change['new']
            bar.style = {'bar_color': f"rgb({255-int(change['new']*2.55)}, {255-int(change['new']*2.55)}, {255-int(change['new']*2.55)})"}
            pl.value = f"{change['new']}%"
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        progress_slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        box = widgets.VBox([
            title,
            widgets.Label(value="진행률", layout=label_layout),
            progress_bar,
            progress_slider,
            progress_label,
            widgets.Label(value="마감일", layout=label_layout),
            due_label,
            widgets.Label(value="중요도", layout=label_layout),
            priority_label,
            button_row
        ], layout=widgets.Layout(border='1px solid lightgray', padding='10px', margin='10px 0'))

        todo_list_box.children += (box,)

# 입력 위젯
title_input = widgets.Text(placeholder='할 일 입력', layout=full_width)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=full_width)
due_input = widgets.DatePicker(layout=full_width)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=full_width)

def add_task(_):
    if title_input.value:
        todo_items.append({
            'title': title_input.value,
            'progress': progress_input.value,
            'due': due_input.value or date.today(),
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button = widgets.Button(description='추가', layout=full_width, button_style='success')
add_button.on_click(add_task)

# 저장 및 불러오기
def save_data(_):
    with open('todo_data.json', 'w') as f:
        json.dump([{**i, 'due': i['due'].isoformat()} for i in todo_items], f)

def load_data(_):
    try:
        with open('todo_data.json', 'r') as f:
            todo_items.clear()
            for i in json.load(f):
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': date.fromisoformat(i['due']),
                    'priority': i['priority']
                })
        update_overall_progress()
        refresh_list()
    except Exception as e:
        print("불러오기 실패:", e)

save_btn = widgets.Button(description='저장', layout=half_width, button_style='info')
load_btn = widgets.Button(description='불러오기', layout=half_width, button_style='warning')
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 출력 구성
display(widgets.VBox([
    widgets.HTML("<h3>📂 DAVID mk27</h3>"),
    title_input,
    progress_input,
    due_input,
    priority_input,
    add_button,
    widgets.HBox([save_btn, load_btn], layout=full_width),
    widgets.Label("최종 진행률", layout=widgets.Layout(justify_content='center')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=full_width),
    todo_list_box
]))

update_overall_progress()
refresh_list()
